<a href="https://colab.research.google.com/github/wmok12/Thesis-research-minimumwage-employment/blob/main/Data_Organizing_and_Cleaning_construction_industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages and loading data

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
pip install ipumspy

In [ ]:
from ipumspy import readers, ddi
# Loading data
ddi_codebook = readers.read_ipums_ddi("/content/drive/MyDrive/Thesis/Voorbereiding/cps_00008.xml")
raw_ipums_df_codes = readers.read_microdata(ddi_codebook, "/content/drive/MyDrive/Thesis/Voorbereiding/cps_00008.csv.gz")


/usr/local/lib/python3.10/dist-packages/ipumspy/readers.py:70: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See the `ipums_conditions` attribute of this codebook for terms of use.
See the `ipums_citation` attribute of this codebook for the appropriate citation.
  warnings.warn(


# Explore data

In [ ]:
print(raw_ipums_df_codes.head())

   YEAR  SERIAL  MONTH  STATEFIP  AGE  SEX  RACE  MARST  EMPSTAT  LABFORCE  \
0  2010       1      1         1   60    2   100      4       10         2   
1  2010       1      1         1   29    1   100      6       21         2   
2  2010       6      1         1   85    2   100      4       36         1   
3  2010       7      1         1   52    2   100      4       21         2   
4  2010       7      1         1   37    1   100      4       21         2   

    IND  CLASSWKR  UHRSWORK1  EDUC  HOURWAGE  
0  8190        22         40    91    999.99  
1   170        22        999    73    999.99  
2     0         0        999    92    999.99  
3  7580        22        999   111    999.99  
4   770        22        999    91    999.99  


In [ ]:
raw_ipums_df = raw_ipums_df_codes.copy()

In [ ]:
# Checking features
unique_values = raw_ipums_df["RACE"].unique()
print(unique_values)

value_counts = raw_ipums_df["RACE"].value_counts()
print(value_counts)

<IntegerArray>
[100, 200, 651, 806, 802, 801, 803, 807, 814, 300, 652, 805, 810, 808, 812,
 809, 813, 804, 820, 811, 830, 816, 818, 817, 819, 815]
Length: 26, dtype: Int64
100    15529698
200     2041471
651      984867
300      247547
802      131173
801      127037
652       90365
803       80406
804       18606
805       17446
813       14822
809       14429
810       13478
806        6402
811        2697
812        2082
807        1967
830        1489
808        1402
820        1134
817         847
814         732
816         703
815         558
818         286
819         159
Name: RACE, dtype: Int64


# Coding variable into words in stead of standard coding from IPUMS CPS

In [ ]:
# Mapping, transforming coding into words
mappings = {
    'SEX': {1: 'Male', 2: 'Female', 9: 'NIU'},
    'EMPSTAT': {10: 'Employed', 21: 'Unemployed', 36: 'Unemployed', 32: 'Unemployed', 34: 'Unemployed', 0: 'Others', 12: 'Employed', 1: 'Others', 22: 'Unemployed'},
    'LABFORCE': {0: 'Others', 1: 'Not labor force', 2: 'In labor force'},
    'IND': {770: 'Construction', 7860: 'Elementary and secondary schools'},
    'WKSTAT': {11: 'Full-time', 50: 'Unemployed, seeking for part-time', 99: 'Others', 12: 'Full-time', 41: 'Part-time', 21: 'Full-time', 22: 'Part-time', 60: 'Unemployed, seeking for part-time', 42: 'Part-time', 13 :'Full-time', 15: 'Part-time', 14: 'Part-time'},
    'EDUC': {91: 'AC degree, occupational program', 73: 'High school diploma', 92: 'AC degree, academic program', 111: 'Bachelor\'s degree', 50: 'Grade 10', 60: 'Grade 11', 20: 'Grades 5 or 6', 81: 'Some college', 40: 'Grade 9', 1: 'No schooling', 30: 'Grades 7 or 8', 10: 'Grades 1 t/m 4', 123: 'Master/s degree', 125: 'Doctorate degree', 124: 'Professional school degree', 71: 'Grade 12', 2: 'No schooling'},
    'STATEFIP': {12: 'Florida'},
    'CLASSWKR': {22: 'Work for wages', 0: 'Others', 13: 'Self-employed', 25: 'Work for wages', 23: 'Work for wages', 28: 'Work for wages', 27:'Work for wages', 14: 'Self-employed', 26: 'Work for wages', 29: 'Others'},
    'MARST': {6: 'Never married', 1: 'Married', 5: 'Others', 4:'Others', 3: 'Others', 2: 'Married'},
    'RACE': {100: 'White', 200: 'BAME', 651: 'BAME', 300: 'Others', 802: 'Others', 805: 'Others', 801: 'Others', 652: 'Others', 810: 'Others', 803: 'Others', 804: 'Others', 806: 'Others'}
}

# Inserting mapping to the dataframe
for column, mapping in mappings.items():
    if column in raw_ipums_df.columns:
        raw_ipums_df[column] = raw_ipums_df[column].map(mapping)

# Checking the dataframe
print(raw_ipums_df.head())

   YEAR  SERIAL  MONTH STATEFIP  AGE     SEX   RACE          MARST  \
0  2010       1      1      NaN   60  Female  White         Others   
1  2010       1      1      NaN   29    Male  White  Never married   
2  2010       6      1      NaN   85  Female  White         Others   
3  2010       7      1      NaN   52  Female  White         Others   
4  2010       7      1      NaN   37    Male  White         Others   

      EMPSTAT         LABFORCE           IND        CLASSWKR  UHRSWORK1  \
0    Employed   In labor force           NaN  Work for wages         40   
1  Unemployed   In labor force           NaN  Work for wages        999   
2  Unemployed  Not labor force           NaN          Others        999   
3  Unemployed   In labor force           NaN  Work for wages        999   
4  Unemployed   In labor force  Construction  Work for wages        999   

                              EDUC  HOURWAGE  
0  AC degree, occupational program    999.99  
1              High school diploma

# Filter on state and industry

In [ ]:
# Filter on STATEFIP - Florida (12)

flt_ipums_df = raw_ipums_df[raw_ipums_df["STATEFIP"] == 'Florida']

value_counts = flt_ipums_df["STATEFIP"].value_counts()
print(value_counts)

Florida    810886
Name: STATEFIP, dtype: int64


In [ ]:
# Filter on IND - Construction industry (770)

flt_ipums_df = flt_ipums_df[flt_ipums_df["IND"] == 'Construction']

value_counts = flt_ipums_df["IND"].value_counts()
print(value_counts)


Construction    30613
Name: IND, dtype: int64


# Removing unnecessary columns

In [ ]:
# Remove IND - STATEFIP - SERIAL (they are not needed)

flt_ipums_df = flt_ipums_df.drop(columns=["IND", "STATEFIP", "SERIAL", "HOURWAGE", 'UHRSWORK1'])
print(flt_ipums_df.head())

     YEAR  MONTH  AGE   SEX   RACE          MARST     EMPSTAT        LABFORCE  \
530  2010      1   29  Male  White  Never married    Employed  In labor force   
537  2010      1   55  Male   BAME  Never married  Unemployed  In labor force   
538  2010      1   58  Male  White        Married    Employed  In labor force   
571  2010      1   44  Male  White         Others    Employed  In labor force   
575  2010      1   60  Male  White        Married  Unemployed  In labor force   

           CLASSWKR                 EDUC  
530  Work for wages    Bachelor's degree  
537  Work for wages  High school diploma  
538  Work for wages  High school diploma  
571  Work for wages         Some college  
575  Work for wages  High school diploma  


# Adding minimumwage to the dataset

In [ ]:
# Adding minimumwage

#
min_wage_data = {
    "YEAR": [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
    "MINIMUM WAGE": [7.25, 7.25, 7.67, 7.79, 7.93, 8.05, 8.05, 8.10, 8.25, 8.46, 8.56, 10.00, 10.00]
}

min_wage_df = pd.DataFrame(min_wage_data)

# Adding the minimum wage to the excisting dataFrame
ipums_df = pd.merge(flt_ipums_df, min_wage_df, on="YEAR", how="left")

# Move the 'MINIMUM_WAGE' to the end
minimum_wage_col = ipums_df.pop("MINIMUM WAGE")
ipums_df.insert(len(ipums_df.columns), "MINIMUM WAGE", minimum_wage_col)

# Checking the results
print(ipums_df)

# Check
unique_values = ipums_df[["YEAR", "MINIMUM WAGE"]].drop_duplicates()

# Print the unique value per year
for index, row in unique_values.iterrows():
    print(f"YEAR {row['YEAR']} = {row['MINIMUM WAGE']}")



       YEAR  MONTH  AGE     SEX   RACE          MARST     EMPSTAT  \
0      2010      1   29    Male  White  Never married    Employed   
1      2010      1   55    Male   BAME  Never married  Unemployed   
2      2010      1   58    Male  White        Married    Employed   
3      2010      1   44    Male  White         Others    Employed   
4      2010      1   60    Male  White        Married  Unemployed   
...     ...    ...  ...     ...    ...            ...         ...   
30608  2022     12   46    Male  White        Married    Employed   
30609  2022     12   20    Male  White  Never married    Employed   
30610  2022     12   43  Female  White         Others    Employed   
30611  2022     12   63  Female  White        Married    Employed   
30612  2022     12   62    Male  White        Married    Employed   

             LABFORCE        CLASSWKR                 EDUC  MINIMUM WAGE  
0      In labor force  Work for wages    Bachelor's degree          7.25  
1      In labor force

In [ ]:
# Checking features
unique_values = ipums_df["MARST"].unique()
print(unique_values)

value_counts = ipums_df["MARST"].value_counts()
print(value_counts)

['Never married' 'Married' 'Others']
Married          17844
Never married     7837
Others            4932
Name: MARST, dtype: int64


# Saving it to csv

In [ ]:
import pandas as pd

df = pd.DataFrame(ipums_df)

df.to_csv('/content/drive/MyDrive/Thesis/Voorbereiding/dataset_cps_construction.csv', index=False)



# Cleaning outliers

In [ ]:
# Data loading
dataset = pd.read_csv('/content/drive/MyDrive/Thesis/Voorbereiding/dataset_cps_construction.csv')
print(dataset.head())

In [ ]:
# Remove outliers based on Z-score between -3 and 3

numerical_features_outliers = ['AGE']

# Identify outliers using Z-score
z_scores = stats.zscore(dataset[numerical_features_outliers])
outliers = (abs(z_scores) > 3).any(axis=1)

print("Number of outliers:", outliers.sum())

cleaned_dataset= dataset[~outliers]

# Print the cleaned dataset shape for verification
print("Original dataset shape:", dataset.shape)
print("Cleaned dataset shape:", cleaned_dataset.shape)

# Print the cleaned dataset's descriptive statistics
print(cleaned_dataset.describe())

In [ ]:
cleaned_dataset = cleaned_dataset[cleaned_dataset['CLASSWKR'] != 'Others']

In [ ]:
df = pd.DataFrame(cleaned_dataset)

df.to_csv('/content/drive/MyDrive/Thesis/Voorbereiding/cleaned_dataset_cps_construction.csv', index=False)
